In [2]:
import numpy as np
import pandas as pd

# pip install pytest-warnings
import warnings
warnings.filterwarnings(action='ignore')

In [101]:
data_path = ''

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [102]:
# 한글화
sales_train = sales_train.rename(columns={'date':'날짜','date_block_num': '월ID', 
                                        'shop_id':'상점ID', 'item_id':'상품ID',
                                         'item_price':'판매가', 
                                        'item_cnt_day':'판매량'})
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [103]:
shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

shops.head()

,상점명,상점ID
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [104]:
items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})

items.head()

,상품명,상품ID,상품분류ID
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [105]:
item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품분류명',
                                          'item_category_id': '상품분류ID'})

item_categories.head()

,상품분류명,상품분류ID
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [106]:
test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

test.head()

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


# 다운캐스팅 : 작은 데이터 타입으로 변환

In [107]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col]=df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col]=pd.to_numeric(df[col], downcast = 'integer')
        else:
            df[col]=pd.to_numeric(df[col], downcast='float')
    end_mem= df.memory_usage().sum()/1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100*(start_mem - end_mem) / start_mem))
        
    return df

In [108]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df=downcast(df)

62.5% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


In [109]:
test.head()

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


# 데이터 조합 생성

In [110]:
from itertools import product

In [111]:
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.000000,1
1,03.01.2013,0,25,2552,899.000000,1
2,05.01.2013,0,25,2552,899.000000,-1
3,06.01.2013,0,25,2554,1709.050049,1
4,15.01.2013,0,25,2555,1099.000000,1


In [112]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   날짜      object 
 1   월ID     int8   
 2   상점ID    int8   
 3   상품ID    int16  
 4   판매가     float32
 5   판매량     int16  
dtypes: float32(1), int16(2), int8(2), object(1)
memory usage: 50.4+ MB


In [113]:
sales_train['월ID'].unique().size

34

In [114]:
t1 = sales_train[sales_train['월ID'] == 0]
# t2 = t1['상점ID'].unique()
t1[t1['상점ID'] == t1['상점ID'].unique()[0]]


,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.0,1
40084,10.01.2013,0,59,22151,399.0,1
77502,04.01.2013,0,59,5603,699.0,1
77503,19.01.2013,0,59,5587,199.0,2
77504,31.01.2013,0,59,5613,5571.0,1
...,...,...,...,...,...,...
80427,23.01.2013,0,59,1471,2599.0,1
80428,27.01.2013,0,59,35,399.0,1
80429,24.01.2013,0,59,32,349.0,1
80430,03.01.2013,0,59,32,349.0,1


In [115]:
# max_num=0
# for i in range(sales_train['월ID'].unique().size):
#     t1 = sales_train[sales_train['월ID'] == i]
#     for j in t1['상점ID'].unique():
#         k = t1[t1['상점ID'] == j].shape[0]
#         if k > max_num:
#             max_num = k

# max_num
sales_train['상점ID'].unique().size

60

In [116]:
# max_num=0
# for i in range(sales_train['월ID'].unique().size):
#     t1 = sales_train[sales_train['월ID'] == i]
#     for j in t1['상품ID'].unique():
#         k = t1[t1['상품ID'] == j].shape[0]
#         if k > max_num:
#             max_num = k

# max_num
sales_train['상품ID'].unique().size

21807

In [117]:
l = 0
for i in range(sales_train['월ID'].unique().size):
    j = sales_train[sales_train['월ID'] == i]
    k = j['상점ID'].shape[0] * sales_train['상품ID'].unique().size
    l += k
l
# sales_train['상품ID'].unique().size

64022059143

In [118]:
sales_train.loc[sales_train['월ID']==0,'상점ID'].unique()

array([59, 25, 24, 23, 19, 22, 18, 21, 28, 27, 29, 26,  4,  6,  2,  3,  7,
        0,  1, 16, 15,  8, 10, 14, 13, 12, 53, 31, 30, 32, 35, 56, 54, 47,
       50, 42, 43, 52, 51, 41, 38, 44, 37, 46, 45], dtype=int8)

In [119]:
sales_train['월ID']

0           0
1           0
2           0
3           0
4           0
           ..
2935844    33
2935845    33
2935846    33
2935847    33
2935848    33
Name: 월ID, Length: 2935849, dtype: int8

In [120]:
# 월ID 고윳값별 모든 상점ID 고윳값, 상품ID 고윳값을 구해 조합
train=[]

for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
#     print(all_shop[0:5])
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
#     print(all_item[0:5])
    train.append(np.array(list(product([i], all_shop, all_item))))
#    print(train[0:5])
                               
idx_features = ['월ID', '상점ID', '상품ID']
# print(type(train))
train = pd.DataFrame(np.vstack(train), columns=idx_features)

train

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10913845,33,21,7635
10913846,33,21,7638
10913847,33,21,7640
10913848,33,21,7632


In [122]:
sales_train[['월ID', '상점ID', '상품ID']]

,월ID,상점ID,상품ID
0,0,59,22154
1,0,25,2552
2,0,25,2552
3,0,25,2554
4,0,25,2555
...,...,...,...
2935844,33,25,7409
2935845,33,25,7460
2935846,33,25,7459
2935847,33,25,7440


In [126]:
test_agg = sales_train.head(3)
test_agg

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.0,1
1,03.01.2013,0,25,2552,899.0,1
2,05.01.2013,0,25,2552,899.0,-1


In [127]:
train_agg =[]

for i in test_agg['월ID'].unique():
    all_shop = test_agg.loc[test_agg['월ID']==i, '상점ID'].unique()
#     print(all_shop[0:5])
    all_item = test_agg.loc[test_agg['월ID']==i, '상품ID'].unique()
#     print(all_item[0:5])
    train_agg.append(np.array(list(product([i], all_shop, all_item))))
#    print(train[0:5])
                               
idx_features = ['월ID', '상점ID', '상품ID']
# print(type(train))
train_agg = pd.DataFrame(np.vstack(train_agg), columns=idx_features)

train_agg

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,25,22154
3,0,25,2552


# 월간 판매량 추가

In [130]:
group = sales_train.groupby(idx_features).agg({'판매량':'sum'})
group = group.reset_index()
group = group.rename(columns={'판매량':'월단 판매량'})
group

,월ID,상점ID,상품ID,월단 판매량
0,0,0,32,6
1,0,0,33,3
2,0,0,35,1
3,0,0,43,1
4,0,0,51,2
...,...,...,...,...
1609119,33,59,22087,6
1609120,33,59,22088,2
1609121,33,59,22091,1
1609122,33,59,22100,1


In [131]:
# train, group 병합
train = train.merge(group, on=idx_features, how='left')
train

,월ID,상점ID,상품ID,월단 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
10913845,33,21,7635,NaN
10913846,33,21,7638,NaN
10913847,33,21,7640,NaN
10913848,33,21,7632,NaN


In [135]:
# 가비지 컬렉션 : 할당한 메모리 중 더는 사용하지 않는 영역 해제

import gc

del group
gc.collect()

NameError: name 'group' is not defined

In [136]:
# test 데이터 이어붙이기

In [137]:
test['월ID']=34

In [138]:
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)

In [139]:
all_data = all_data.fillna(0)
all_data

,월ID,상점ID,상품ID,월단 판매량
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0
...,...,...,...,...
11128045,34,45,18454,0.0
11128046,34,45,16188,0.0
11128047,34,45,15757,0.0
11128048,34,45,19648,0.0


In [140]:
# 나머지 데이터 병합
all_data = all_data.merge(shops,on='상점ID', how='left')
all_data = all_data.merge(items,on='상품ID', how='left')
all_data = all_data.merge(item_categories,on='상품분류ID', how='left')

all_data = downcast(all_data)

del shops, items, item_categories
gc.collect();

26.4% 압축됨


In [141]:
all_data.head()

,월ID,상점ID,상품ID,월단 판매량,상점명,상품명,상품분류ID,상품분류명
0,0,59,22154,1,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,0,59,2552,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,0,59,2554,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
3,0,59,2555,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства
4,0,59,2564,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео


In [142]:
all_data = all_data.drop(['상점명', '상품명', '상품분류명'], axis=1)

In [143]:
# train, validation, test 데이터 구분

In [145]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월단 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월단 판매량'], axis=1)
# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월단 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월단 판매량']
y_train = y_train.clip(0, 20) # 타깃값을 0 ~ 20로 제한
# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월단 판매량']
y_valid = y_valid.clip(0, 20)

In [146]:
del all_data
gc.collect();

# 모델 훈련 및 성능 검증

In [148]:
# LightGBM : 고윳값 개수가 너무 많은 범주형 데이터는 수치형 데이터롤 취급해야 성능이 더 잘나온다고 함.

In [151]:
import lightgbm as lgb
# pip install lightgbm

params = {'metric':'rmse',
         'num_leaves':255,
         'learning_rate':0.01,
         'force_col_wise':True,
         'random_state':10}

#범주형 피쳐 설정
cat_features = ['상점ID', '상품분류ID']

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

In [152]:
lgb_model = lgb.train(params=params,
                     train_set=dtrain,
                      num_boost_round=500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      verbose_eval=50)

[LightGBM] [Info] Total Bins 426
[LightGBM] [Info] Number of data points in the train set: 10675678, number of used features: 4
[LightGBM] [Info] Start training from score 0.299125
[50]	training's rmse: 1.14777	valid_1's rmse: 1.06755
[100]	training's rmse: 1.11425	valid_1's rmse: 1.0386
[150]	training's rmse: 1.09673	valid_1's rmse: 1.02671
[200]	training's rmse: 1.08573	valid_1's rmse: 1.02027
[250]	training's rmse: 1.07722	valid_1's rmse: 1.01661
[300]	training's rmse: 1.0698	valid_1's rmse: 1.0138
[350]	training's rmse: 1.06317	valid_1's rmse: 1.01084
[400]	training's rmse: 1.05734	valid_1's rmse: 1.00936
[450]	training's rmse: 1.05224	valid_1's rmse: 1.00818
[500]	training's rmse: 1.04792	valid_1's rmse: 1.00722


In [155]:
preds = lgb_model.predict(X_test).clip(0,20)

In [156]:
# 타겟값 0~20 범위 제한

In [157]:
submission['item_cnt_month']=preds
submission.to_csv('submission.csv', index=False)

In [158]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();